No Place for Hate

We seek to find the correlation of the occurrence of Hate Crimes and:
* Education
* Poverty
* Unemployment Rates




In [4]:
#Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress


In [40]:
# import and merge data sets - Education, and Unemployment Rates by state
#read Education csv file and review the sample rows
edu_data = pd.read_csv("Resources/EducationReport.csv")
#create a new dataframe with only needed columns for years 2014-2018
edu_data_df = pd.DataFrame(edu_data)
edu_data_mycolm = edu_data_df[["Unnamed: 0", "Total.4", "Urban.4", "Rural.4"]]
#renamed the columns to have more meaningful names
edu_data_upd = edu_data_mycolm.rename(columns={"Unnamed: 0" : "area_name", "Total.4" : "Total Edu 2014-18", "Urban.4" : "Total Edu Urban 2024-18", "Rural.4" : "Total Edu Rural 2014-18"})
#dropped the first row of the data set with column titles we are not using
edu_data_final = edu_data_upd.drop([0,1], axis=0)
edu_data_final.head(3)
#note that I haven't removed the "null" data from the data set yet; they need to be removed from the merged table

,area_name,Total Edu 2014-18,Total Edu Urban 2024-18,Total Edu Rural 2014-18
2,Alabama,24.90%,28.00%,15.20%
3,Alaska,29.20%,31.80%,23.90%
4,Arizona,28.90%,29.60%,15.80%


In [ ]:
unempl_data = pd.read_csv("Resources/Unemployment.csv", encoding="ISO-8859-1")

In [68]:
unempl_data["Average Unempl 2014-18"] =(unempl_data["Unemployment_rate_2018"] + unempl_data["Unemployment_rate_2017"]
+ unempl_data["Unemployment_rate_2016"] + unempl_data["Unemployment_rate_2015"] + 
unempl_data["Unemployment_rate_2014"])/4
unempl_data.head(3)

,FIPStxt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018,Average Unempl 2014-18
0,0,US,United States,NaN,NaN,NaN,"142,601,667","136,904,680","5,696,987",4.0,...,"155,102,319","6,286,707",3.9,"163,100,055","157,115,247","5,984,808",3.7,"61,937",NaN,6.100
1,1000,AL,Alabama,NaN,NaN,NaN,"2,133,223","2,035,594","97,629",4.6,...,"2,130,845","85,782",3.9,"2,241,747","2,174,483","67,264",3.0,"49,881",100.0,6.725
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,"21,720","20,846",874,4.0,...,"25,261",935,3.6,"26,172","25,458",714,2.7,"59,338",119.0,5.850


In [70]:
#filter data to get to the state level (the columns where rural urban code & metro 2013 columns are empty)
unempl_data['Rural_urban_continuum_code_2013'].replace('', np.nan, inplace=True)
unempl_state_interim = unempl_data[unempl_data['Rural_urban_continuum_code_2013'].isnull()]
unempl_state_interim
unempl_data_clean = unempl_state_interim[unempl_state_interim['Metro_2013']!=0]
unempl_data_clean.head(3)

,FIPStxt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018,Average Unempl 2014-18
0,0,US,United States,NaN,NaN,NaN,"142,601,667","136,904,680","5,696,987",4.0,...,"155,102,319","6,286,707",3.9,"163,100,055","157,115,247","5,984,808",3.7,"61,937",NaN,6.100
1,1000,AL,Alabama,NaN,NaN,NaN,"2,133,223","2,035,594","97,629",4.6,...,"2,130,845","85,782",3.9,"2,241,747","2,174,483","67,264",3.0,"49,881",100.0,6.725
69,2000,AK,Alaska,NaN,NaN,NaN,"319,511","299,146","20,365",6.4,...,"329,901","22,761",6.5,"347,779","326,430","21,349",6.1,"74,912",100.0,8.550


In [72]:
#select columns needed for years 2014-2018
unempl_data_mycolm = unempl_data_clean[["area_name", "Average Unempl 2014-18"]]
unempl_data_mycolm.head(3)

,area_name,Average Unempl 2014-18
0,United States,6.100
1,Alabama,6.725
69,Alaska,8.550


In [73]:
# Kamran personal poverty work
load_poverty = ("Resources/PovertyReport.csv")
poverty_df = pd.read_csv(load_poverty)
poverty_df = poverty_df.drop(columns={"Textbox98", "Textbox99", "total_est_pct_Bmin2",
                                     "total_est_pct_Bmax2", "under18_pct_Bmin2", "under18_pct_Bmax2"})
poverty_df = poverty_df.rename(columns={"state_National": "area_name",
                                       "total_est_pct2": "Poverty (2018)",
                                       "under18_pct2": "Under 18 in Poverty (2018)"})
poverty_df.head()

,area_name,Poverty (2018),Under 18 in Poverty (2018)
0,Alabama,16.8,23.9
1,Alaska,11.1,14.5
2,Arizona,14.1,20.4
3,Arkansas,16.8,23.8
4,California,12.8,17.4


In [45]:
poverty_df["Poverty (2018)"] = poverty_df["Poverty (2018)"].map("{:.2f}%".format)
poverty_df["Under 18 in Poverty (2018)"] = poverty_df["Under 18 in Poverty (2018)"].map("{:.2f}%".format)
poverty_df.head()

,area_name,Poverty (2018),Under 18 in Poverty (2018)
0,Alabama,16.80%,23.90%
1,Alaska,11.10%,14.50%
2,Arizona,14.10%,20.40%
3,Arkansas,16.80%,23.80%
4,California,12.80%,17.40%


In [36]:
hate_crime = ("Resources/hate_crimes538.csv")
hate_crime_df = pd.read_csv(hate_crime)
hate_crime_df.head(3)

,state,median_household_income,share_unemployed_seasonal,share_population_in_metro_areas,share_population_with_high_school_degree,share_non_citizen,share_white_poverty,gini_index,share_non_white,share_voters_voted_trump,hate_crimes_per_100k_splc,avg_hatecrimes_per_100k_fbi
0,Alabama,42278,0.060,0.64,0.821,0.02,0.12,0.472,0.35,0.63,0.125839,1.806410
1,Alaska,67629,0.064,0.63,0.914,0.04,0.06,0.422,0.42,0.53,0.143740,1.656700
2,Arizona,49254,0.063,0.90,0.842,0.10,0.09,0.455,0.49,0.50,0.225320,3.413928


In [49]:
hate_crime_colm = hate_crime_df[["state", "median_household_income", "hate_crimes_per_100k_splc"]]
hate_crime_colm = hate_crime_colm.rename(columns={"state" : "area_name"})
hate_crime_colm.head(3)

,area_name,median_household_income,hate_crimes_per_100k_splc
0,Alabama,42278,0.125839
1,Alaska,67629,0.143740
2,Arizona,49254,0.225320


In [74]:
# Merge the merged Demographic Data with Hate Crimes 358 by state
merged_df = pd.merge(edu_data_final,unempl_data_mycolm, on="area_name")
merged_df.head(3)

,area_name,Total Edu 2014-18,Total Edu Urban 2024-18,Total Edu Rural 2014-18,Average Unempl 2014-18
0,Alabama,24.90%,28.00%,15.20%,6.725
1,Alaska,29.20%,31.80%,23.90%,8.550
2,Arizona,28.90%,29.60%,15.80%,6.950


In [75]:
merged_df1 = pd.merge(merged_df, poverty_df, on="area_name")
merged_df1.head(3)

,area_name,Total Edu 2014-18,Total Edu Urban 2024-18,Total Edu Rural 2014-18,Average Unempl 2014-18,Poverty (2018),Under 18 in Poverty (2018)
0,Alabama,24.90%,28.00%,15.20%,6.725,16.8,23.9
1,Alaska,29.20%,31.80%,23.90%,8.550,11.1,14.5
2,Arizona,28.90%,29.60%,15.80%,6.950,14.1,20.4


In [76]:
merged_df_fin = pd.merge(merged_df1,hate_crime_colm, on="area_name")
merged_df_fin.head(3)

,area_name,Total Edu 2014-18,Total Edu Urban 2024-18,Total Edu Rural 2014-18,Average Unempl 2014-18,Poverty (2018),Under 18 in Poverty (2018),median_household_income,hate_crimes_per_100k_splc
0,Alabama,24.90%,28.00%,15.20%,6.725,16.8,23.9,42278,0.125839
1,Alaska,29.20%,31.80%,23.90%,8.550,11.1,14.5,67629,0.143740
2,Arizona,28.90%,29.60%,15.80%,6.950,14.1,20.4,49254,0.225320


In [ ]:
# occurrence of Hate Crimes to Education (Bachelor's Degree vs. High School) by State

In [ ]:
# occurrence of Hate Crimes to Poverty Level by State

In [ ]:
# occurrence of Hate Crimes to Poverty Level by State

In [ ]:
# Clean FBI Data set - take 2014-2018 data and average hate crimes fbi